<a href="https://colab.research.google.com/github/hieunguyen-cyber/AI_Project_CBH_GHHKM/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Huấn luyện mô hình nhận diện biển báo giao thông sử dụng YOLOv10**



Nhóm thực hiện: CBH_GHHKM

**Lưu ý**: Trước khi thực hiện, chuyển đổi môi trường chạy sang GPU. Nhớ kiểm tra thời gian còn lại của GPU có đủ cho thời gian huấn luyện dự kiến.

---

###**Bước 1**: Clone các file cần thiết

Ở đây, YOLOv10 sẽ được clone từ GitHub và tập dữ liệu sử dụng để huấn luyện sẽ được tải về sử dụng API từ Kaggle.

Nguồn:
* [Vietnamese Traffic Signs Dataset](https://www.kaggle.com/datasets/maitam/vietnamese-traffic-signs)
* [YOLOv10](https://github.com/THU-MIG/yolov10)

In [ ]:
!git clone https://github.com/hieunguyen-cyber/API-key.git
!git clone https://github.com/THU-MIG/yolov10

###**Bước 2**: Tiến hành tải về tập dữ liệu

Ở đây tôi đã thêm 1 file **sign.yaml** vào tập dữ liệu gốc nhằm phục vụ cho việc huấn luyện.

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/API-key/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle
!kaggle datasets download -d hieunguyen2208/vietnamese-traffic-signs

Tiến hành giải nén tập dữ liệu

In [ ]:
!unzip /content/vietnamese-traffic-signs.zip

###**Bước 3**: Tiến hành cài đặt YOLOv10

**Lưu ý**: Do lỗi dependencies giữa một vài thư viện của PyTorch nên bước này chạy hai lần

In [ ]:
%cd yolov10
!pip install torch
!pip install -q -r requirements.txt
!pip install -e
!wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt

###**Bước 4**: Tiến hành chia tập dư liệu thành tập train và val

Cụ thể, tập dữ liệu gốc gồm 3216 cặp image-label, ta chia thành hai tập, tập train gồm 2572 cặp, tập val gồm 644 cặp

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_dir = '/content/dataset/images'
label_dir = '/content/dataset/labels'
train_image_dir = '/content/dataset/images/train'
val_image_dir = '/content/dataset/images/val'
train_label_dir = '/content/dataset/labels/train'
val_label_dir = '/content/dataset/labels/val'

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

if len(images) == 0:
    raise ValueError("No images found in the dataset directory.")
if len(images) < 5:
    raise ValueError("Not enough samples to split. Ensure you have at least 5 images.")

train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, train_size=0.8, random_state=42
)

for img, lbl in zip(train_images, train_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(train_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(train_label_dir, lbl))

for img, lbl in zip(val_images, val_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(val_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(val_label_dir, lbl))


###**Bước 5**: Tiến hành huấn luyện

Ở đây, do hạn chế về mặt phần cứng và để tiết kiệm thời gian huấn luyện, đội quyết định chọn các thông số như sau cho lần đầu:
* EPOCHS = 200
* BATCH_SIZE = 256
* IMAGE_SIZE = 320

Sau đó, file trọng số **best.pt** được trích xuất và huấn luyện lại nhiều lần với thông số mỗi lần là:
* EPOCHS = 30 - 50
* BATCH_SIZE = 256
* IMAGE_SIZE = 320

Cụ thể, với kích cỡ của tập dữ liệu như trên, thời gian huấn luyện 1 epoch trung bình mất 45s.

Nếu là lần đầu huấn luyện

In [ ]:
from ultralytics import YOLOv10
MODEL_PATH = '/content/yolov10/yolov10n.pt'
model = YOLOv10(MODEL_PATH)
YAML_PATH = '/content/dataset/sign.yaml'
EPOCHS = 200
IMG_SIZE = 320
BATCH_SIZE = 256
model.train(data=YAML_PATH ,
            epochs= EPOCHS ,
            batch=BATCH_SIZE,
            imgsz= IMG_SIZE)

Nếu là huấn luyện lại

* Tải về file **best.pt** từ lần huấn luyện gần nhất

In [ ]:
%cd /content
!git clone https://github.com/hieunguyen-cyber/Weight_AIBK_2024.git
!mv /content/Weight_AIBK_2024/best.pt /content

* Tiến hành huấn luyện

In [ ]:
%cd /content/yolov10
from ultralytics import YOLOv10
MODEL_PATH = '/content/best.pt'
model = YOLOv10(MODEL_PATH)
YAML_PATH = '/content/dataset/sign.yaml'
EPOCHS = 1
IMG_SIZE = 320
BATCH_SIZE = 256
model.train(data=YAML_PATH ,
            epochs= EPOCHS ,
            batch=BATCH_SIZE,
            imgsz= IMG_SIZE)

###**Bước 6**: Lưu file **best.pt** lên repository GitHub để lưu trữ

* **Bước 1**: Tải file về máy

In [ ]:
from google.colab import files
files.download('/content/yolov10/runs/detect/train/weights/best.pt')

* **Bước 2**: Tải file lên Github sau : [Weight_AIBK_2024](https://github.com/hieunguyen-cyber/Weight_AIBK_2024)
